In [3]:
import pandas as pd
import numpy as np

iris_df = pd.read_csv('iris_clusters.csv', sep=';')
iris_df.describe()

,pl,pw,sl,sw,id
count,300.000000,300.000000,300.000000,300.000000,300.000000
mean,5.909333,3.018000,3.686333,1.223000,7989.316667
std,2.675814,0.447598,1.890350,0.764451,4277.531346
min,4.100000,1.900000,0.900000,0.100000,87.000000
25%,5.075000,2.775000,1.500000,0.400000,4034.500000
50%,5.700000,3.000000,4.300000,1.400000,8347.500000
75%,6.400000,3.300000,5.000000,1.900000,11716.000000
max,50.000000,4.500000,16.300000,2.600000,14950.000000
